# Tower Coverage Optimization

The decision variables are
$x_{ij}=\begin{cases}1\\ 0\end{cases}$ if tower is built in region $(i,j)$, otherwise.

$y_{ij}=\begin{cases}1\\ 0\end{cases}$ if region $(i,j)$ is covered by at least one tower, otherwise.

and the given information includes

$H=$ height of city

$W=$ width of city

$T=$ maximum number of towers

$p_{ij}=$ population in region $(i,j)$.

The integer linear programming formulation of this problem becomes

### maximize $z = \sum_{i=1}^{H}\sum_{j=1}^{W}p_{ij}y_{ij}$

### subject to:

$x_{ij}+\sum_{(k,l)\in\text{neighbors}(i,j)}x_{kl}\geq y_{ij} \quad i=1,2,...,H; \ j=1,2,...,W$

$\sum_{i=1}^{H}\sum_{j=1}^{W}x_{ij}\leq T$

$x_{ij}\in\{0,1\} \quad i=1,2,...,H; \ j=1,2,...,W$

$y_{ij}\in\{0,1\} \quad i=1,2,...,H; \ j=1,2,...,W$

In [9]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

In [10]:
def mixed_integer_linear_programming(direction, A, senses, b, c, l, u, types):
    # create an empty optimization problem
    prob = cp.Cplex()

    # add decision variables to the problem including their coefficients in objective and ranges
    prob.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist(), types = types.tolist())

    # define problem type
    if direction == "maximize":
        prob.objective.set_sense(prob.objective.sense.maximize)
    else:
        prob.objective.set_sense(prob.objective.sense.minimize)

    # add constraints to the problem including their directions and right-hand side values
    prob.linear_constraints.add(senses = senses.tolist(), rhs = b.tolist())

    # add coefficients for each constraint
    row_indices, col_indices = A.nonzero()
    prob.linear_constraints.set_coefficients(zip(row_indices.tolist(), col_indices.tolist(), A.data.tolist()))

    print(prob.write_as_string())
    # solve the problem
    prob.solve()
    
    # check the solution status
    print(prob.solution.get_status())
    print(prob.solution.status[prob.solution.get_status()])

    # get the solution
    x_star = prob.solution.get_values()
    obj_star = prob.solution.get_objective_value()

    return(x_star, obj_star)


In [11]:
def cell_tower_coverage_problem(populations_file, T):

    populations = np.loadtxt(populations_file)
    E = populations.size
    H = populations.shape[0]
    W = int(E/H)
    l = np.repeat(0, 2*E)
    u = np.repeat(1, 2*E)
    c = np.concatenate((np.repeat(0, E), [populations[i,j].astype(int) for i in range(H) for j in range(W)]))
    b = np.concatenate((np.repeat(0, E), [T]))
    senses = np.concatenate((np.repeat("G",E), ["L"]))
    types = np.repeat("B", 2*E)
                        #sol_üst_köşe  #üst_ara                 #sağ_üst_köşe    #soldan_sağa_tarama                                                                 #sol_alt_köşe    #alt_ara                 #sağ_alt_köşe  #son_row
    aij = np.concatenate(([1,1,1,-1], np.tile([1,1,1,1,-1], W-2), [1,1,1,-1], np.tile(np.concatenate(([1,1,1,1,-1], np.tile([1,1,1,1,1,-1], W-2),[1,1,1,1,-1])), H-2), [1,1,1,-1], np.tile([1,1,1,1,-1], W-2), [1,1,1,-1], np.repeat(1,E))) 
    
    row = np.concatenate(( np.repeat(0,4), np.concatenate(([np.repeat(i+1,5) for i in range(W-2)])).tolist(), (np.repeat(W-1,4)), 
        np.concatenate(([np.concatenate((np.repeat((j+1)*W,5), np.concatenate(([np.repeat(i+1+((j+1)*W), 6) for i in range(W-2)])).tolist(),
        np.repeat(((j+2)*W)-1,5))) for j in range(H-2)])).tolist(), np.repeat((H-1)*W,4), 
        np.concatenate(([np.repeat(((H-1)*W)+i+1, 5) for i in range(W-2)])).tolist(), np.repeat(E-1,4), np.repeat(E, E) ))
    
    col = np.concatenate(( (0,1,W,E), np.concatenate(([(0+i,1+i,2+i,W+1+i,E+1+i) for i in range(W-2)])).tolist(), (W-2,W-1,(2*W)-1,E+W-1),
                     np.concatenate(([np.concatenate(( (j*W,(j+1)*W,((j+1)*W)+1,(j+2)*W,(j+(H+1))*W), 
                               np.concatenate(([(1+i+(j*W),W+i+(j*W),W+1+i+(j*W),W+2+i+(j*W),(2*W)+1+i+(j*W),((H+1)*W)+1+i+(j*W)) for i in range(W-2)])), 
                                    (((j+1)*W)-1,((j+2)*W)-2,((j+2)*W)-1,((j+3)*W)-1,((j+W)*W)-1) )) for j in range(H-2)])).tolist(),
                      ((H-2)*W,(H-1)*W,(H-1)*W+1,((2*E)-1)-(H+1)),
                      np.concatenate(([(((H-2)*W)+1+i,(H-1)*W+i,((H-1)*W)+1+i,((H-1)*W)+2+i,(2*E)-H-1+i) for i in range(W-2)])).tolist(),
                      (((H-2)*W)+1+H,((H-1)*W)+H ,((H-1)*W)+H+1, (2*E)-1),
                      [i for i in range(0,E)]))
    
    A = sp.csr_matrix((aij, (row, col)), shape = (E+1, 2*E))
    x_star, obj_star = mixed_integer_linear_programming("maximize", A, senses, b, c, l, u, types)
    X_star = np.array(x_star[:E]).reshape(H,W)
    return(X_star)

In [12]:
X_star = cell_tower_coverage_problem("populations.txt", 3)
print(X_star)

Default variable names x1, x2 ... being created.
Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 obj1: 732 x25 + 539 x26 + 949 x27 + 508 x28 + 806 x29 + 881 x30 + 646 x31
       + 757 x32 + 257 x33 + 630 x34 + 994 x35 + 547 x36 + 105 x37 + 859 x38
       + 876 x39 + 589 x40 + 615 x41 + 345 x42 + 136 x43 + 370 x44 + 433 x45
       + 419 x46 + 631 x47 + 485 x48
Subject To
 c1:  x1 + x2 + x7 - x25 >= 0
 c2:  x1 + x2 + x3 + x8 - x26 >= 0
 c3:  x2 + x3 + x4 + x9 - x27 >= 0
 c4:  x3 + x4 + x5 + x10 - x28 >= 0
 c5:  x4 + x5 + x6 + x11 - x29 >= 0
 c6:  x5 + x6 + x12 - x30 >= 0
 c7:  x1 + x7 + x8 + x13 - x31 >= 0
 c8:  x2 + x7 + x8 + x9 + x14 - x32 >= 0
 c9:  x3 + x8 + x9 + x10 + x15 - x33 >= 0
 c10: x4 + x9 + x10 + x11 + x16 - x34 >= 0
 c11: x5 + x10 + x11 + x12 + x17 - x35 >= 0
 c12: x6 + x11 + x12 + x18 - x36 >= 0
 c13: x7 + x13 + x14 + x19 - x37 >= 0
 c14: x8 + x13 + x14 + x15 + x20 - x38 >= 0
 c15: x9 + x14 + x15 + x16 + x21 - x39 >= 0
 c16: x10 + x15 + x16 + x17 + x22 - x40 >= 0
 c17: x11 + x16 + x17 + x18 + x23 - x41 >= 0
 c18: x12 + x